# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторайзеров. Сравните метрики и выберите победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [152]:
import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from razdel import tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [4]:
# разделение данных на train и test в соотношении 90/10

train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [8]:
y_train = train.toxic.values
y_test = test.toxic.values

In [6]:
# векторизация со стандартной токенизацией

default_vectorizer = CountVectorizer()
X_def = default_vectorizer.fit_transform(train.comment)
X_test_1 = default_vectorizer.transform(test.comment)

In [7]:
# векторизация с токенизацией razdel

def cust_tokenizer(text):
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    return(tokens)

cust_vectorizer = CountVectorizer(tokenizer=cust_tokenizer)
X_cust = cust_vectorizer.fit_transform(train.comment)
X_test_2 = cust_vectorizer.transform(test.comment)

In [15]:
# берем логистическую регрессию и учим ее сначала на дефолтных векторах

clf = LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000).fit(X_def, y_train)

pred_1 = clf.predict(X_test_1)

In [17]:
print(classification_report(y_test, pred_1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.82      0.87       961
         1.0       0.71      0.85      0.77       481

    accuracy                           0.83      1442
   macro avg       0.81      0.84      0.82      1442
weighted avg       0.85      0.83      0.84      1442



In [18]:
# а потом на векторах с токенизацией раздела

clf = LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000).fit(X_cust, y_train)

pred_2 = clf.predict(X_test_2)

In [19]:
print(classification_report(y_test, pred_2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.83      0.87       961
         1.0       0.72      0.85      0.78       481

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.82      1442
weighted avg       0.85      0.84      0.84      1442



Метрики для обоих классификаторов практически одинаковые. Однако в репорте видно, что классификатор с токенизацией razdel все же чуть лучше по практически каждому параметру.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [ ]:
# ваш код

## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [26]:
vect1 = CountVectorizer(max_df=0.6, min_df=8, ngram_range=(1, 2), encoding='utf-8', tokenizer=cust_tokenizer, max_features=8000)
vect2 = TfidfVectorizer(max_df=0.4, min_df=6, ngram_range=(1, 2), tokenizer=cust_tokenizer, max_features=6000)

X1 = vect1.fit_transform(train.comment)
X_test1 = vect1.transform(test.comment) 

X2 = vect2.fit_transform(train.comment)
X_test2 = vect2.transform(test.comment)

In [31]:
clf = MultinomialNB(alpha=1.0, fit_prior=False)
clf.fit(X1, y_train) 
preds = clf.predict(X_test1)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87       961
         1.0       0.72      0.82      0.77       481

    accuracy                           0.83      1442
   macro avg       0.81      0.83      0.82      1442
weighted avg       0.84      0.83      0.84      1442



In [35]:
clf = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf.fit(X2, y_train)
preds = clf.predict(X_test2)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.80      0.83      0.82       961
         1.0       0.64      0.59      0.61       481

    accuracy                           0.75      1442
   macro avg       0.72      0.71      0.72      1442
weighted avg       0.75      0.75      0.75      1442



In [37]:
# функция для выделения 10 самых токсичных текстов

probs = clf.predict_proba(X_test1)

def tox(probs):
    probs = [p[1] for p in probs]
    sort???[:10]
    

SyntaxError: invalid syntax (3025171876.py, line 7)

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

## LogisticRegression

In [49]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/daria/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [106]:
train, test = train_test_split(data, shuffle=True)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

y = train.toxic.values
y_test = test.toxic.values

vectorizer = CountVectorizer(max_df=0.6, min_df=8, ngram_range=(1, 2), max_features=8000, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

clf = LogisticRegression(C=0.1, class_weight='balanced').fit(X, y)

importance = clf.coef_[0]

In [107]:
importance = {i: v for i, v in enumerate(importance)}

In [108]:
[vectorizer.get_feature_names_out()[i] for i, _ in sorted(importance.items(), reverse=True, key=lambda x: x[1])][:5]

['хохлы', 'хохлов', 'тебе', 'дебил', 'нахуй']

## Decision Trees

In [114]:
clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(X, y)

importance = {i: v for i, v in enumerate(clf.feature_importances_)}

In [115]:
[vectorizer.get_feature_names_out()[i] for i, _ in sorted(importance.items(), reverse=True, key=lambda x: x[1])][:5]

['тебе', 'хохлы', 'очень', 'нахуй', 'хохлов']

## Naive Bayes

In [150]:
vectorizer = TfidfVectorizer(max_df=0.6, min_df=8, ngram_range=(1, 2), max_features=8000, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

clf = MultinomialNB(alpha=1.2, fit_prior=False)
clf.fit(X, y)

importance = {i: v for i, v in enumerate(clf.coef_[0])}

/home/daria/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


In [151]:
[vectorizer.get_feature_names_out()[i] for i, _ in sorted(importance.items(), reverse=True, key=lambda x: x[1])][:5]

['это', 'тебе', 'просто', 'хохлы', 'почему']

## Random Forest

In [153]:
vectorizer = TfidfVectorizer(max_df=0.6, min_df=8, ngram_range=(1, 2), max_features=8000, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

clf = RandomForestClassifier(class_weight='balanced')
clf.fit(X, y)

importance = {i: v for i, v in enumerate(clf.feature_importances_)}

In [154]:
[vectorizer.get_feature_names_out()[i] for i, _ in sorted(importance.items(), reverse=True, key=lambda x: x[1])][:5]

['это', 'очень', 'тебе', 'хохлы', 'хохлов']